This program implements a thread-safe Java program that counts the number of occurrences of events numbered from 0 to some max. Multiple threads can call count and frequency simultaneously. The solution is implemented as a monitor such that only one thread can enter count.

In [1]:
%%writefile frequencycounter.java
import java.util.Random;
class FrequencyCounter {
    int[] occurences;
    
    // Class Invariant: ∀ i ∈ 0 ... max - 1 | occurences[i] ≥ 0
    FrequencyCounter(int max) {
        // Create a frequency counter for events numbered 0 to max - 1.
        // Precondition: max > 0
        this.occurences = new int[max];
    }
    
    // Precondition: 0 ≤ event ≤ max - 1
    void count(int event) {
        // Increment the frequency of event by 1.
        synchronized (this) {occurences[event]++;}
    }
    
    // Precondition: 0 ≤ event ≤ max - 1
    int frequency(int event) {
        // Return the frequency of event since creation.
        synchronized (this) {return occurences[event];}
    }
}
class Eventer extends Thread {
    FrequencyCounter fc;
    Eventer(FrequencyCounter fc) {
        this.fc = fc;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < 20000; i++) {
            fc.count(r.nextInt(10));
        }
    }
}
class TestFrequencyCounter {
    public static void main(String[] args) {
        FrequencyCounter fc = new FrequencyCounter(10);
        Eventer ev[] = new Eventer[1000];
        for (int i = 0; i < 1000; i++) ev[i] = new Eventer(fc);
        long startTime = System.currentTimeMillis();
        for (int i = 0; i < 1000; i++) ev[i].start();
        for (int i = 0; i < 1000; i++) {
            try {ev[i].join();} catch (Exception e) {}
        }
        long endTime = System.currentTimeMillis();
        System.out.println(endTime - startTime);
    }
}

Overwriting frequencycounter.java


In [2]:
!javac frequencycounter.java

In [3]:
!java TestFrequencyCounter

7511


This program implements the `FrequencyCounter` as above in the same way except it is more efficient in that it allows for concurrent updates to different events. 

In [4]:
%%writefile highfrequencycounter.java
import java.util.Random;
class HighFrequencyCounter extends FrequencyCounter {
    int[] occurences;
    Object[] eventLocks;
    
    // Class Invariant: ∀ i ∈ 0 ... max - 1 | occurences[i] ≥ 0
    HighFrequencyCounter(int max) {
        // Precondition: max > 0
        super(max);
        this.occurences = new int[max];
        this.eventLocks = new Object[max];
        for (int i = 0; i < max; i++) {this.eventLocks[i] = new Object();}
    }
    
    // Precondition: 0 ≤ event ≤ max - 1
    void count(int event) {
        synchronized (eventLocks[event]) {this.occurences[event]++;}
    }
}
class HEventer extends Thread {
    HighFrequencyCounter hfc;
    HEventer(HighFrequencyCounter hfc) {
        this.hfc = hfc;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < 20000; i++) {
            hfc.count(r.nextInt(10));
        }
    }
}
class TestHighFrequencyCounter {
    public static void main(String[] args) {
        HighFrequencyCounter hfc = new HighFrequencyCounter(10);
        HEventer hev[] = new HEventer[1000];
        for (int i = 0; i < 1000; i++) hev[i] = new HEventer(hfc);
        long hstartTime = System.currentTimeMillis();
        for (int i = 0; i < 1000; i++) hev[i].start();
        for (int i = 0; i < 1000; i++) {
            try {hev[i].join();} catch (Exception e) {}
        }
        long hendTime = System.currentTimeMillis();
        System.out.println(hendTime - hstartTime);
    }
}

Overwriting highfrequencycounter.java


In [5]:
!javac highfrequencycounter.java

In [6]:
!java TestHighFrequencyCounter

3379
